Ref: https://dongzeli95s-organization.gitbook.io/swe-interview-handbook/company-tags/openai/spreadsheet

## Part 1 basic Spreadsheet:

Supporting APIs:

- getCell(string)
- setCell(string, string, int)

In [3]:
# Assume a cell has two children, x and y, and a value.
# The value of the cell is the sum of the values of x and y plus its own value.
class Cell:
    def __init__(self, x: str = None, y: str = None, value: int = 0):
        self.x = x
        self.y = y
        self.value = value

In [17]:
import collections

class Spreadsheet:
    def __init__(self):
        self.child = collections.defaultdict(Cell) # str -> Cell

    def setCell(self, string: str, x: str = None, y: str = None, value: int = 0) -> None:
        self.child[string] = Cell(x, y, value)
    
    # Use dfs or bfs
    def getCell(self, string) -> int:
        if string not in self.child:
            raise ValueError("key %s not found" % string)
        # return self._dfs(self.child[string])
        return self._bfs(self.child[string])

    def _dfs(self, cell):
        res = cell.value
        if cell.x is not None:
            res += self.getCell(cell.x)
        if cell.y is not None:
            res += self.getCell(cell.y)
        return res

    def _bfs(self, cell):
        queue = collections.deque([cell])
        res = 0
        while queue:
            current = queue.popleft()
            res += current.value
            if current.x is not None:
                queue.append(self.child[current.x])
            if current.y is not None:
                queue.append(self.child[current.y])
        return res

In [18]:
# Test cases
spreadsheet = Spreadsheet()
spreadsheet.setCell("A1", value = 5)
spreadsheet.setCell("B1", value = 6)
spreadsheet.setCell("C1", x = "A1", y = "B1", value = 1)
assert(spreadsheet.getCell("C1") == 12)  # Should be 12 (5 + 6 + 1)
spreadsheet.setCell("D1", x = "C1", y = "A1", value = 10)
assert(spreadsheet.getCell("D1") == 27)  # Should be 27 (12 + 5 + 10)

print("All test cases passed!")

All test cases passed!


In [19]:
# But it cannot handle cycles
spreadsheet.setCell("A2", x = "B1", y = "C1", value = 2)
spreadsheet.setCell("B1", x = "A2", y = "A1", value = 3)
# This will cause an infinite loop if we try to getCell("A2") or "B1"
# print(spreadsheet.getCell("A2"))  # Uncommenting this will cause an infinite loop

## Part 2 Spreadsheet handles cycles

In [ ]:
# Detect cycle using colors (https://www.geeksforgeeks.org/dsa/detect-cycle-direct-graph-using-colors/)
# Change setCell to handle cycles and return boolean

import collections

class SpreadsheetWithCycleDetection:
    def __init__(self):
        self.child = collections.defaultdict(Cell) # str -> Cell

    def setCell(self, string: str, x: str = None, y: str = None, value: int = 0) -> bool:
        self.child[string] = Cell(x, y, value)
        # Set cell value when no cycle is detected
        return not self._hasCycle(string, {k: 0 for k in self.child})
    
    # Use dfs or bfs
    def getCell(self, string) -> int:
        if string not in self.child:
            raise ValueError("key %s not found" % string)
        # return self._dfs(self.child[string])
        return self._bfs(self.child[string])

    def _dfs(self, cell):
        res = cell.value
        if cell.x is not None:
            res += self.getCell(cell.x)
        if cell.y is not None:
            res += self.getCell(cell.y)
        return res

    def _bfs(self, cell):
        queue = collections.deque([cell])
        res = 0
        while queue:
            current = queue.popleft()
            res += current.value
            if current.x is not None:
                queue.append(self.child[current.x])
            if current.y is not None:
                queue.append(self.child[current.y])
        return res

    def _hasCycle(self, start, color):
        # 0: unvisited, 1: visiting, 2: visited
        color[start] = 1
        for neighbor in [self.child[start].x, self.child[start].y]:
            if neighbor is not None:
                if color[neighbor] == 1:
                    return True
                if color[neighbor] == 0 and self._hasCycle(neighbor, color):
                    return True
        color[start] = 2
        return False

In [25]:
# Same test cases
spreadsheet = SpreadsheetWithCycleDetection()
assert(spreadsheet.setCell("A1", value = 5))
assert(spreadsheet.setCell("B1", value = 6))
assert(spreadsheet.setCell("C1", x = "A1", y = "B1", value = 1))
assert(spreadsheet.getCell("C1") == 12)  # Should be 12 (5 + 6 + 1)
assert(spreadsheet.setCell("D1", x = "C1", y = "A1", value = 10))
assert(spreadsheet.getCell("D1") == 27)  # Should be 27 (12 + 5 + 10)

assert(spreadsheet.setCell("A2", x = "B1", y = "C1", value = 2))
assert(not spreadsheet.setCell("B1", x = "A2", y = "A1", value = 3))
print("All test cases passed!")

All test cases passed!


Now we want to add the feature to update the cell, and optimize in a way that getCell is O(1).

When a cell is updated, all parent cells need updates as well, so we need another reverse map to traverse from child to parent.

## Part 3 Spreadsheet supports updates

In [6]:
# Add value map so that getCell is O(1), update in setCell
# Add parent map to traverse and update parent values in setCell

import collections

class SpreadsheetWithCycleDetectionAndUpdates:
    def __init__(self):
        self.child = collections.defaultdict(Cell) # str -> Cell
        self.value = collections.defaultdict(int)
        self.parents = collections.defaultdict(set) # str -> set of str

    def setCell(self, string: str, x: str = None, y: str = None, value: int = 0) -> bool:
        cur_value = value + (self.value[x] if x is not None else 0) + (self.value[y] if y is not None else 0)
        cur_child = []
        if x is not None:
            cur_child.append(x)
        if y is not None:
            cur_child.append(y)
        # Add new parent relationships, and check for cycles
        for c in cur_child:
            self.parents[c].add(string)
        # Contains cycle, return False
        if self._hasCycle(string, {k: 0 for k in self.parents}):
            for c in cur_child:
                self.parents[c].remove(string)
            return False
        # Add for the first time
        if string not in self.child:
            self.child[string] = Cell(x, y, value)
            self.value[string] = cur_value
            # print(self.child, self.value, self.parents)
            return True
        
        # Existing key, remove old parent relationships
        prev_value = self.value[string]
        pre_child = []
        if self.child[string].x is not None:
            pre_child.append(self.child[string].x)
        if self.child[string].y is not None:
            pre_child.append(self.child[string].y)
        for p in pre_child:
            self.parents[p].remove(string)
        
        # Update parent values
        self._updateParentValues(string, cur_value - prev_value)
        return True
    
    # O(1) now.
    def getCell(self, string) -> int:
        if string not in self.child or string not in self.value:
            raise ValueError("key %s not found" % string)
        return self.value[string]

    def _dfs(self, cell):
        res = cell.value
        if cell.x is not None:
            res += self.getCell(cell.x)
        if cell.y is not None:
            res += self.getCell(cell.y)
        return res

    def _bfs(self, cell):
        queue = collections.deque([cell])
        res = 0
        while queue:
            current = queue.popleft()
            res += current.value
            if current.x is not None:
                queue.append(self.child[current.x])
            if current.y is not None:
                queue.append(self.child[current.y])
        return res

    def _hasCycle(self, start, color):
        # 0: unvisited, 1: visiting, 2: visited
        # Now update base on parent relationships
        color[start] = 1
        for neighbor in self.parents[start]:
            if neighbor is not None:
                if color[neighbor] == 1:
                    return True
                if color[neighbor] == 0 and self._hasCycle(neighbor, color):
                    return True
        color[start] = 2
        return False
    
    def _updateParentValues(self, string, value):
        # bfs
        queue = collections.deque([string])
        while queue:
            current = queue.popleft()
            self.value[current] += value
            for parent in self.parents[current]:
                queue.append(parent)

        # Or dfs
        # stack = [string]
        # while stack:
        #     current = stack.pop()
        #     self.value[current] += value
        #     for parent in self.parents[current]:
        #         stack.append(parent)

In [ ]:
# Test cases
spreadsheet = SpreadsheetWithCycleDetectionAndUpdates()

# Update value -> value
assert(spreadsheet.setCell("A1", value = 5))
assert(spreadsheet.setCell("B1", value = 6))
assert(spreadsheet.setCell("C1", x = "A1", y = "B1", value = 1))
assert(spreadsheet.getCell("C1") == 12)  # Should be 12 (5 + 6 + 1)

# Cycle detection
assert(spreadsheet.setCell("A2", x = "B1", y = "C1", value = 2))
assert(not spreadsheet.setCell("B1", x = "A2", y = "A1", value = 3))

# Update value -> node
assert(spreadsheet.setCell("A3", value = 3))
assert(spreadsheet.setCell("A4", value = 4))
assert(spreadsheet.setCell("A5", value = 5))
assert(spreadsheet.setCell("A6", value = 6))

# Prev: A34 = A3 + A4 + 10
assert(spreadsheet.setCell("A34", x = "A3", y = "A4", value = 10))
assert(spreadsheet.getCell("A34") == 17)  # Should be 17 (3 + 4 + 10)
assert(spreadsheet.setCell("A4", x = "A5", y = "A6", value = 20))
assert(spreadsheet.getCell("A4") == 31)  # Should be 31 (5 + 6 + 20)
# Now A34 = A3 + A4 + 10 = A3 + (A5 + A6 + 20) + 10
assert(spreadsheet.getCell("A34") == 44)  # Should be 44 (3 + 31 + 10)

print("All test cases passed!")

All test cases passed!
